## Setup

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
from supabase import create_client, Client
import pandas as pd
import dotenv
import os
import uuid
from datetime import datetime, timezone
import requests
from urllib.parse import urlparse, parse_qs



SERVICE_ACCOUNT_KEY = "./../serviceAccountKey.json"
FILE_PATH = "./../Documentation\support new.xlsx"
RESIZED_IMAGES_DIR = r"C:/Users/Anirudh/Desktop/resized"
NEW_IMAGES_DIR = r"C:/Users/Anirudh/Desktop/new"

dotenv.load_dotenv()

True

In [2]:
cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
firebase_admin.initialize_app(cred)
db = firestore.client()
supabase = create_client(os.getenv('EXPO_PUBLIC_SUPABASE_URL'), os.getenv('EXPO_PUBLIC_SUPABASE_ANON_KEY'))

In [3]:
df = pd.read_excel(FILE_PATH)

## Preprocess

In [4]:
df.head()

,Timestamp,Email address,Full Name,Designation,Date of Birth,Sex,Photograph,Spouse's Name,Wedding Anniversary,Phone,...,Business Address,Business Website,About your business,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Phone,T Shirt Size,Shirt Size,Meal Preference,Blood Group
0,2024-12-19 10:03:05,gopi@fin1solutions.com,R GOPINATH,SETS SECRETARY,1972-02-25 00:00:00,Male,https://drive.google.com/open?id=18w_j4W8cQtuI...,VIJAYALAKSHMI,1998-11-02,9841061758,...,NO 264 B BLOCK FLAT NO 4 ANBUR COLONY ANNA NAG...,www.fin1solutions.com,Fin 1 Solutions is a DSA for HDFC Bank for the...,TARUN GOPINATH,SON,7358543242,L,42,Non-Vegetarian,NaN
1,2024-12-26 19:22:13,rtn.uma1505@gmail.com,Rtn. Uma Yuvaraj,Committee Member - Events,1980-05-15 00:00:00,Female,https://drive.google.com/open?id=1_zb_c7fblsNC...,Rtn. Yuvaraj,2020-03-24,9361455621,...,.,notapplicable.com,..,Yuvaraj,Spouse,9361455621,M,0,Vegetarian,NaN
2,2024-12-26 19:35:51,advocatesuresh763@gmail.com,Sureshselvakumar,Committee Member,1965-05-31 00:00:00,Male,https://drive.google.com/open?id=1Yrt2omdhAWWu...,Subbulakshmi,1965-05-31,9840960291,...,"FOM, 4 th floor, Jain citadel Apartment, Justi...",NaN,"My father is retired Judge, I handle all type ...",Subbulakshmi,Wife,9790708712,XXXL,48,Non-Vegetarian,NaN
3,2024-12-26 19:38:22,nokiasai@gmail.com,M Sesha Sai,Co-Chairman Training,1965-08-22 00:00:00,Male,https://drive.google.com/open?id=1dK5I2dsGoxeW...,M Kamala,1989-05-19,9940055555,...,62 Veerabhadran street. Nungambakkam. Chennai ...,www.farmguru.org,Farm Guru is a social initiative that educates...,M Kamala,Wife,9840546655,L,42,Non-Vegetarian,NaN
4,2024-12-27 11:10:49,bharatmpujara@gmail.com,Bharat Pujara,Committee Member,1955-04-15 00:00:00,Male,https://drive.google.com/open?id=1wZj4WhcrnMn1...,Bharati pujara,1976-02-07,9840277772,...,6/7 5th street T K salai Mylapore Chennai 600004,NaN,Suppling Raw material to paper mills for Recyc...,Bharati Pujara,Wife,9840938880,XXXL,50,Vegetarian,NaN


In [6]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_').str.replace("'", "")
df.rename(columns={
    "if_self-employed_:_business_name_/_if_salaried_:_company_name": "company_name",
    "designation": "sets_designation",
    "role_in_business_/_designation_in_company": "designation",
    "full_name": "name"}, inplace=True)
df.drop(columns=['timestamp'], inplace=True)

In [7]:
df["name_lower"] = df["name"].str.lower()
df["company_name_lower"] = df["company_name"].str.lower()
df["club_name_lower"] = df["club_name"].str.lower()
df["priority"] = 0
df['role'] = "member"
df["support"] = True
df.fillna("", inplace=True)

In [8]:
def convert_timestamp_to_string(timestamp):
    
    if pd.isna(timestamp):
        return ""
    
    if isinstance(timestamp, datetime):
        if timestamp.tzinfo is None:  # Check if the timestamp is naive
            timestamp = timestamp.replace(tzinfo=timezone.utc)
        else:
            timestamp = timestamp.astimezone(timezone.utc)
    formatted_string = timestamp.strftime('%Y-%m-%d %H:%M:%S%z')
    if formatted_string.endswith("+0000"):
        formatted_string = formatted_string[:-2] + ":00"
    return formatted_string

In [9]:
def transform_date(value):
    # Check if the value is in DD/MM format or full date format
    if not value:
        return ""
    
    try:
        # Attempt to parse full date
        date = pd.to_datetime(value, format='%d/%m/%Y')
        return date.strftime('%d/%m/%Y')
    except ValueError:
        # If ValueError occurs, assume the value is in DD/MM format
        current_year = datetime.now().year
        return f"{value}/{current_year}"

In [10]:
df.columns

Index(['email_address', 'name', 'sets_designation', 'date_of_birth', 'sex',
       'photograph', 'spouses_name', 'wedding_anniversary', 'phone', 'email',
       'club_name', 'rotarian_since', 'rotary_foundation_title',
       'residential_address', 'company_name', 'designation',
       'type_of_business', 'business_address', 'business_website',
       'about_your_business', 'emergency_contact_name',
       'emergency_contact_relationship', 'emergency_contact_phone',
       't_shirt_size', 'shirt_size', 'meal_preference', 'blood_group',
       'name_lower', 'company_name_lower', 'club_name_lower', 'priority',
       'role', 'support'],
      dtype='object')

In [11]:
datetime_columns = ['date_of_birth', 'wedding_anniversary']

In [12]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
# df['wedding_anniversary'] = pd.to_datetime(df['wedding_anniversary'], errors='coerce')
df['wedding_anniversary'] = df['wedding_anniversary'].apply(transform_date)
df['wedding_anniversary'] = pd.to_datetime(df['wedding_anniversary'], errors='coerce', dayfirst=True)


In [13]:
for col in datetime_columns:
    df[col] = df[col].dt.tz_localize("UTC", ambiguous='NaT', nonexistent='NaT')

In [14]:
for col in datetime_columns:
    df[col] = df[col].apply(convert_timestamp_to_string)

In [15]:
emails_to_filter = ['bharatmpujara@gmail.com', 'kamalaselvam@gmail.com']
df = df[df['email'].isin(emails_to_filter)]

In [16]:
df

,email_address,name,sets_designation,date_of_birth,sex,photograph,spouses_name,wedding_anniversary,phone,email,...,t_shirt_size,shirt_size,meal_preference,blood_group,name_lower,company_name_lower,club_name_lower,priority,role,support
4,bharatmpujara@gmail.com,Bharat Pujara,Committee Member,1955-04-15 00:00:00+00:00,Male,https://drive.google.com/open?id=1wZj4WhcrnMn1...,Bharati pujara,1976-02-07 00:00:00+00:00,9840277772,bharatmpujara@gmail.com,...,XXXL,50,Vegetarian,,bharat pujara,global waste recyclers pvt ltd,rotary club of madras,0,member,True
13,kamalaselvam@gmail.com,Kamala Selvam,Co-Chairman Spouse,1976-05-10 00:00:00+00:00,Female,https://drive.google.com/open?id=1kg9vcbuL8o_k...,Selvam Alaghhappan,1994-01-23 00:00:00+00:00,9092123456,kamalaselvam@gmail.com,...,M,38,Non-Vegetarian,,kamala selvam,"business - pink lotus event studio, chettinad ...",rotary club of madras southwest,0,member,True


In [17]:
full_data = df.to_dict(orient='records')

## Upload Image + Return Public URL

In [18]:
def download_file_from_google_drive_upload_supabase(uid, photo_url):       
    parsed_url = urlparse(photo_url)
    query_params = parse_qs(parsed_url.query)
    file_id = query_params['id'][0]
    
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": file_id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    return save_response_content(uid, response)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith("download_warning"):
            return value

    return None

def save_response_content(uid, response):   
    file_name = f"{uid}.png"  # Assuming the image format is JPEG
    file_content = response.content
    
    try:
        responseSupabase = supabase.storage.from_("members").upload(file_name, file_content)
        public_url = supabase.storage.from_("members").get_public_url(file_name)
        return public_url
    except Exception as e:
        print(e)


In [19]:
def upload_image_from_local(uid, folder_path):
    file_name = f"{uid}.png"
    file_path = os.path.join(folder_path, file_name)

    if not os.path.exists(file_path):
        print(f"File {file_name} not found in {folder_path}.")
        return None

    try:
        # Open the file in binary read mode
        with open(file_path, "rb") as file:
            file_content = file.read()
        
        # Upload the file to Supabase
        responseSupabase = supabase.storage.from_("members").upload(file_name, file_content)
        public_url = supabase.storage.from_("members").get_public_url(file_name)
        
        return public_url
    except Exception as e:
        print(f"An error occurred while uploading {file_name}: {e}")
        return None

def download_file_from_google_drive_save_local(uid, photo_url, save_folder):       
    parsed_url = urlparse(photo_url)
    query_params = parse_qs(parsed_url.query)
    file_id = query_params['id'][0]
    
    URL = "https://docs.google.com/uc?export=download&confirm=1"

    session = requests.Session()

    response = session.get(URL, params={"id": file_id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {"id": file_id, "confirm": token}
        response = session.get(URL, params=params, stream=True)

    os.makedirs(save_folder, exist_ok=True)

    # Define the file path
    file_path = os.path.join(save_folder, f"{uid}.png")

    # Save the downloaded file to the specified folder
    try:
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(32768):
                file.write(chunk)
        print(f"File saved locally as {file_path}")
    except Exception as e:
        print(f"Error saving file: {e}")

    return None

## Upsert Users

- Check for email existence in firebase auth, if it exists then update record in firestore
- If email does not exist then create new user in firebase auth and then create new record in firestore with doc id = user.uid

In [20]:
new_users_list = []

In [21]:
data = [record for record in full_data if record['email'] in new_users_list]

In [22]:
data = full_data

In [24]:
for index, user_data in enumerate(data):
    email = user_data.get("email")
    if not email:
        print(f"Skipping record without email: {user_data}")
        continue
    
    try:
        # Check if user exists in Firebase Auth
        user = auth.get_user_by_email(email)
        print(f"({index}/{len(data)}) User exists: {email} (UID: {user.uid}). Updating record")
        
        # # Update Firestore record
        # user_data['id'] = user.uid
        # try:
        #     # public_url = download_file_from_google_drive(user_data['id'], user_data['photograph'])
        #     public_url = upload_image_from_local(user_data['id'], RESIZED_IMAGES_DIR)
        #     user_data['photograph'] = public_url
            
        #     # Upsert the record into Supabase
        #     response = supabase.table("members").upsert(user_data).execute()

        #     if len(response.data) <= 0:
        #         print(f"Error while upserting record for email {email}: {response}")
        #     else:
        #         print(f"Successfully upserted record for email {email}")

        # except Exception as e:
        #     print(f"An error occurred while processing email {email}: {str(e)}")
        
    except auth.UserNotFoundError:
        print(f"({index}/{len(data)}) User does not exist: {email}")
        
        # Create new user in Firebase Auth
        new_user = auth.create_user(email=email, password=email.split('@')[0] + '_password')
        print(f"Created new user: {email} (UID: {new_user.uid})")
        
        # Create new Firestore record
        user_data['id'] = new_user.uid
        
        new_users_list.append(user_data['email'])
        try:
            public_url = download_file_from_google_drive_upload_supabase(user_data['id'], user_data['photograph'])
            # public_url = download_file_from_google_drive_save_local(user_data['id'], user_data['photograph'], NEW_IMAGES_DIR)
            
            user_data['photograph'] = public_url
            
            response = supabase.table("members").upsert(user_data).execute()

            if len(response.data) <= 0:
                print(f"Error while upserting record for email {email}: {response}")
            else:
                print(f"Successfully upserted record for email {email}")

        except Exception as e:
            print(f"An error occurred while processing email {email}: {str(e)}")


(0/2) User does not exist: bharatmpujara@gmail.com
Created new user: bharatmpujara@gmail.com (UID: LyfEgwK2f3cw852OixKtGItWZ3u1)
Successfully upserted record for email bharatmpujara@gmail.com
(1/2) User does not exist: kamalaselvam@gmail.com
Created new user: kamalaselvam@gmail.com (UID: JjF2BOw2vcWOuFa48QxzSvTCUqo1)
Successfully upserted record for email kamalaselvam@gmail.com


In [32]:
new_users_list

[]

In [64]:
for elem in data:
    print(elem['photograph'])

https://drive.google.com/open?id=18w_j4W8cQtuIGQQU92Z2aEpB2BKjA9WU
https://drive.google.com/open?id=1_zb_c7fblsNC6Khm8DYcDXZTXGr2it0f
https://drive.google.com/open?id=1Yrt2omdhAWWuzSH7e4DzJGNKrFsfZwM7
https://drive.google.com/open?id=1dK5I2dsGoxeWS4FvyB8tqeqfW9Pkqk9o
https://drive.google.com/open?id=1wZj4WhcrnMn10_8RxTJcAPBZQU-4tI6k
https://drive.google.com/open?id=1HtZeulncNUBn5-rv8U_LzBnkBHOMmEmn
https://drive.google.com/open?id=15qiI9FCy1vxvZ3pdkkYOKFwo7Bm7r0IO
https://drive.google.com/open?id=1-JqK3qXJs_-mUyEMKeK8v3EboaHcb9zw
https://drive.google.com/open?id=1t2A22XhfabjywzW1VIsx34qohC3YQEUX
https://drive.google.com/open?id=1f5fSxkR67SNhRYdMDRi5Sc8IOqO5AZsa
https://drive.google.com/open?id=1wfhKDzzyundvJo3WWFiwVRE0hZyFkYFI
https://drive.google.com/open?id=1GGLSnkyBcRfTbhGmJRJ7M6fihaOYjG1h
https://drive.google.com/open?id=1IoKUIX0hEb95uAbGnoskEr0X9aJgjYL5
https://drive.google.com/open?id=1kg9vcbuL8o_kcsDR75a8LCh4seEtSTfR
https://drive.google.com/open?id=15PUAvIWuTeMwsfgEybBFqp_l5YWo